# Dataset: *ferguson*

- df.to_csv(r'D:\論文\PHEME9\Data\CSV\ferguson-df.csv',index=False)
- src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ferguson-src.csv',index=False)

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ferguson-df.csv', encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ferguson-src.csv', encoding = 'utf-8', header =0)
print(df.shape)
print(src_tw_df.shape)

(23165, 11)
(1143, 7)


In [3]:
df.isnull().sum()

src_tweet_id           0
src_user_id            0
src_tweet              0
src_date               0
reply_tweet_id       133
reply_user_id        133
reply_tweet          133
reply_date           133
label                  0
cleaned_reply_tw    1436
reply_sentiment        0
dtype: int64

In [7]:
df.loc[df.reply_tweet.isnull(),['src_tweet_id','label']]

,src_tweet_id,label
370,498265555833720832,0
414,498269365175721984,0
415,498269636551798784,0
459,498271531386953728,0
967,498286202077085696,0
...,...,...
21697,500359088757555200,1
21971,500360889347096576,1
22138,500362083221786625,1
22531,500377191029673984,1


In [14]:
df.dropna(inplace = True)
df.reset_index(drop=True,inplace=True)

In [15]:
src_tw_df.isnull().sum()

src_tweet_id       0
src_user_id        0
src_tweet          0
src_date           0
label              0
cleaned_src_tw    21
src_sentiment      0
dtype: int64

In [16]:
src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet

31      : Teenager #MikeBrown won't start college on M...
65      Police dogs snarl at a crowd of mourners in #F...
74      RT @_westwest_yall: Never forget the original ...
158     RT @TOO_LIVE_MILZ: The sad truth RT @kupkakeka...
289     RT @PDPJ: #Ferguson picture from earlier in th...
314     RT @manofsteele: Man picks up burning tear gas...
315     RT @tabithalou: This what they not showing you...
405     RT @AntonioFrench: State Senator asks the #Fer...
436     RT @freegcf: RT @organizemo: @OpFerguson PLS R...
443     RT @JJohnsonKMOV: Intersection of Canfield and...
459     RT .@FOX2now: The #STL SWAT team has weapons d...
466     RT @FOX2now: The #STL SWAT team has weapons dr...
472     RT .@ryanjreilly: A massive SWAT team just sho...
544     This. RT @grasswire: State Senator asks the #F...
562     RT @YourAnonNews: SWAT police just entered a M...
595     @eyeFLOODpanties RT @iFalasteen: The oppressed...
657     RT @Smith83K This Cartoon Says It All: via @Be...
751     HERE A

In [17]:
a = src_tw_df[src_tw_df.cleaned_src_tw.isnull()].src_tweet_id.values
df.loc[df.src_tweet_id.isin(a),['reply_tweet','cleaned_reply_tw']]

,reply_tweet,cleaned_reply_tw
455,@LocaFMCGB @thepoliticalcat How to set filters...,set filter tweet media people warn see
456,@LocaFMCGB I love my bullet lol #NutriBullet #...,love bullet lol nutribullet nutrition health s...
457,@LocaFMCGB @BlueDuPage what happened?!,happen
458,@LocaFMCGB @jblack501c3 unreal talk about exce...,unreal talk excessive force
459,@LocaFMCGB sad.,sad
...,...,...
18973,@BigBoi it takes time to put a story that all...,take time put story officer learn like learn l...
18974,@BigBoi @rolandsmartin So when do police ask r...,police ask robbery suspect walk sidewalk get s...
18975,@BigBoi I'm sooo proud to see people not buyin...,im sooo proud see people not buy lie let keep ...
18976,@BigBoi @rolandsmartin They needed something t...,need something muddy secondrate announcement p...


## Emotion Analysis

### T5 Based
<https://huggingface.co/mrm8488/t5-base-finetuned-emotion?text=I+wish+you+were+here+but+it+is+impossible>

In [ ]:
t5_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion",use_fast=False)
t5_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")



def get_emotion_t5(text):
    input_ids = t5_tokenizer.encode(text , return_tensors='pt')
    output = t5_model.generate(input_ids=input_ids,max_length=2)
  
    dec = [t5_tokenizer.decode(ids) for ids in output]
    label = dec[0]
    return label

In [ ]:
emotion_t5 = df.cleaned_reply_tw.apply(get_emotion_t5)

In [ ]:
df['reply_emo_t5'] = emotion_t5

In [ ]:
import re
df['reply_emo_t5'] = df['reply_emo_t5'].apply(lambda x : re.sub('<pad> ',"",x))
df.reply_emo_t5

In [ ]:
src_emo_t5 = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_t5)
src_tw_df['src_emo_t5'] = src_emo_t5

In [ ]:
src_tw_df['src_emo_t5']  = src_tw_df.src_emo_t5.apply(lambda x : re.sub('<pad> ',"",x))

In [ ]:
ax = df.reply_emotion.value_counts().plot.bar()

for i in ax.patches:
    height = i.get_height()
    ax.text(
        i.get_x() + i.get_width() / 2,height, str(height), ha="center", va="bottom"
    )
plt.xticks(rotation=0)
plt.show()
#plt.savefig('D:/論文/PHEME9/Code/graph/ferguson/rep-emotion.pdf',dpi=300)

### roBERTa-base model
<https://huggingface.co/cardiffnlp/twitter-roberta-base-emotion>

In [4]:
import numpy as np
from scipy.special import softmax

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
ro_tokenizer = AutoTokenizer.from_pretrained(MODEL)
ro_model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [5]:
def get_emotion_roBERT(text):
    
    def preprocess(txt):
        new_text = []
        for t in txt.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    
    labels = ['anger','joy','optimism','sadness']
    text = preprocess(text)
    encoded_input = ro_tokenizer(text, return_tensors='pt')
    output = ro_model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    emo = labels[ranking[0]]
    return emo

In [6]:
emotion_ro = df.cleaned_reply_tw.apply(get_emotion_roBERT)

In [7]:
df['reply_emo_ro'] = emotion_ro
df.to_csv(r'D:\論文\PHEME9\Data\CSV\ferguson-df-2.csv',index=False)

In [8]:
src_emo_ro = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_roBERT)
src_tw_df['src_emo_ro'] = src_emo_ro

In [9]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ferguson-src-2.csv',index=False)

In [ ]:
print("source tweet:", df.src_tweet[9126])
print("original reply tw: ", df.reply_tweet[9126])
print("cleaned reply tw: ", df.cleaned_reply_tw[9126])

print('roBERT based: ',df.reply_emo_ro[9126])
print('t5 based: ',df.reply_emo_t5[9126])
print('text2emotion: ', df.reply_emo_te[9126])